In [ ]:
run Keypoints_RCNN.ipynb

In [ ]:
run 2_Calculating_Angles.ipynb

In [ ]:
run 3_Reading_RPLidar.ipynb

In [ ]:
run 4_Stage_Estimation.ipynb

# Define Environment

In [ ]:
class Environment():
    
    '''
    This wrapper works as an RL-Environment for the Jetson Nano.
    Arguments are: 
        model  -> pytorch model trained in Unity.
        angles -> list of angles required to measure
        FOV    -> How wide the camera lens is. Default = 160
    '''
    
    def __init__(self, model, angles, robot, FOV=160):
        self.previous_readings = {x:0 for x in angles} 
        self.angles = angles
        self.model = model
        self.FOV = FOV
        self.robot = robot
        
        
    def calculate_angle_and_phase(self):
        keypoints, image, counts, objects, peaks = execute({'new': camera.value})
        phase = self.calculate_phase(keypoints)
        if phase[0]==1:
            angle = calculate_angle(WIDTH, keypoints, self.FOV)
        else:
            angle=[0]  # Stage 2 & 3 don't need angle (see CSharp script)
        
        return phase, angle

    
    def calculate_phase(self, keypoints):
        phase = estimating_phase(keypoints)
        return phase
    
    
    def read_lidar(self):
        self.previous_readings = read_lidar_wrapper(self.angles,self.previous_readings)
        
        #normalize lidar readings (max range: 12m) if not norm yet
        self.previous_readings = {k:v/4000 if v>1 else v for k,v in zip(self.previous_readings.keys(), self.previous_readings.values())}
        self.previous_readings = {k:1 if v>1 or v==0 else v for k,v in zip(self.previous_readings.keys(), self.previous_readings.values())}
    
    def observe(self):
        self.read_lidar()
        phase, angle = self.calculate_angle_and_phase()
        
        if isinstance(angle, dict):
            angles = list(angle.values())
        
        else: angles = [0]
        
        observation = phase + angle + list(self.previous_readings.values())
        return observation
    
    
    def sample_action(self, observation):
        observation = torch.Tensor(observation).cuda()
        hidden,_ = self.model.network_body(vis_inputs=[0],vec_inputs=[observation])
        distribution = self.model.distribution(hidden)
        action = distribution.sample()
        return action
    
    
    def step(self,action):
        action = action.cpu().detach().numpy()[0]
        
        speed_move, speed_turn = float(abs(action[0])), float(abs(action[1]))
        
        # backward
        if action[0] < 0:
            robot.backward(speed_move)
            time.sleep(0.5)
            robot.stop()
            
        # forward
        else:
            robot.forward(speed_move)
            time.sleep(0.5)
            robot.stop()
            
        time.sleep(0.2)
            
        # turn left
        if action[1] < 0:
            robot.left(speed_turn)
            time.sleep(0.3)
            robot.stop()
            
        # turn right
        else:
            robot.right(speed_turn)
            time.sleep(0.3)
            robot.stop()


# Load model

In [ ]:
# Input your model name which must be inside the root folder
model = torch.load("George-1-128.pth")
model.eval()

# Define the angles to read using LiDAR

In [ ]:
angles = [x for x in range(0,360,20)]

# Initialize the robot

In [ ]:
robot = Robot()

In [ ]:
env = Environment(model, angles, robot)

In [ ]:
observation = env.observe()
print(observation)
print(len(observation))

In [ ]:
action = env.sample_action(observation)
action

In [ ]:
env.step(action)